## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [9]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


In [10]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [11]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

C:\Users\Orpheus\Anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [12]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32, kernel_size=(3, 3), input_shape=(32,32,3), activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization(momentum=0.99, epsilon=0.01))

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(32, kernel_size=(3, 3), input_shape=(32,32,3), activation='relu'))
classifier.add(BatchNormalization(momentum=0.99, epsilon=0.001))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(output_dim=100, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='relu'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

C:\Users\Orpheus\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=100)`
C:\Users\Orpheus\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=10)`


Epoch 1/100
50000/50000 [==============================] - 44s 879us/step - loss: 2.4662 - accuracy: 0.2095
Epoch 2/100
50000/50000 [==============================] - 47s 931us/step - loss: 2.0486 - accuracy: 0.2888
Epoch 3/100
50000/50000 [==============================] - 46s 916us/step - loss: 1.9921 - accuracy: 0.2972
Epoch 4/100
50000/50000 [==============================] - 46s 916us/step - loss: 1.8567 - accuracy: 0.3573
Epoch 5/100
50000/50000 [==============================] - 46s 911us/step - loss: 1.7263 - accuracy: 0.4173
Epoch 6/100
50000/50000 [==============================] - 46s 916us/step - loss: 1.7337 - accuracy: 0.4277
Epoch 7/100
50000/50000 [==============================] - 46s 916us/step - loss: 1.7632 - accuracy: 0.3846
Epoch 8/100
50000/50000 [==============================] - 46s 917us/step - loss: 1.7202 - accuracy: 0.4298
Epoch 9/100
50000/50000 [==============================] - 46s 919us/step - loss: 1.7041 - accuracy: 0.4423
Epoch 10/100
50000/50000 [==

50000/50000 [==============================] - 46s 917us/step - loss: 1.1540 - accuracy: 0.6237
Epoch 76/100
50000/50000 [==============================] - 46s 917us/step - loss: 1.1825 - accuracy: 0.6148
Epoch 77/100
50000/50000 [==============================] - 46s 917us/step - loss: 1.1311 - accuracy: 0.6276
Epoch 78/100
50000/50000 [==============================] - 46s 917us/step - loss: 1.2187 - accuracy: 0.6108
Epoch 79/100
50000/50000 [==============================] - 46s 915us/step - loss: 1.2354 - accuracy: 0.6062
Epoch 80/100
50000/50000 [==============================] - 46s 912us/step - loss: 1.1252 - accuracy: 0.6358
Epoch 81/100
50000/50000 [==============================] - 46s 917us/step - loss: 1.1136 - accuracy: 0.6371
Epoch 82/100
50000/50000 [==============================] - 46s 916us/step - loss: 1.1410 - accuracy: 0.6250
Epoch 83/100
50000/50000 [==============================] - 46s 916us/step - loss: 1.1038 - accuracy: 0.6378
Epoch 84/100
50000/50000 [======

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [13]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]], dtype=float32)